In [2]:
#  Подключаю необходимые библиотеки
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

In [3]:
import os
os.chdir("/home/vladimir/jupyter/pyDA/Lab_01")


In [4]:
df = pd.read_csv('winequality-red.csv', sep=';')

In [5]:
df.describe(include='all')

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [6]:
df['quality'].value_counts(normalize=True)
#несбалансированная выборка

5    0.425891
6    0.398999
7    0.124453
4    0.033146
8    0.011257
3    0.006254
Name: quality, dtype: float64

In [7]:
#объединим
df['quality'] = df['quality'].replace(to_replace=3, value=4).replace(8, 7).replace(4, 5)


In [8]:
df['quality'].value_counts(normalize=True)
#уже лучше

5    0.465291
6    0.398999
7    0.135710
Name: quality, dtype: float64

In [10]:
X = df.iloc[:, :-1].values  
y = df.iloc[:, 11].values  

#  разделяю на обучающую и тестовую выборку
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)  


In [11]:
from sklearn.ensemble import GradientBoostingClassifier
#  from sklearn.metrics import classification_report
from sklearn import metrics

In [29]:
model = GradientBoostingClassifier(random_state=42,
                                   # Доля наблюдений в случайной подвыборке для очередного дерева
                                   subsample=0.66,
                                   # Доля переменных в случайной подвыборке для очередного дерева
                                   max_features='sqrt', 
                                   # Число деревьев
                                   n_estimators=700,
                                   #  критерий качества ‘deviance’ (кросс-энтропия) или ‘exponential’ (как в AdaBoost)
                                   #  ‘deviance’ для классификации с вероятностями на выходе
                                   loss='deviance', 
                                   # shrinkage На это число умножаем каждое дерево.
                                   # Рекомендуется выставлять небольшие значения из (0, 0.3].
                                   learning_rate=0.01, 
                                   #  загрязнение дерева измеряем “mse” или “friedman_mse”  (mse с улучшениями)
                                    #  “mae” удалено из нынешней версии,  
                                   criterion='friedman_mse', 
                                   # минимальное уменьшение загрязнения 
                                   min_impurity_decrease=0.001, 
                                   # минимальное число наблюдений в узле потомке
                                   min_samples_leaf=3, 
                                   # минимальное число наблюдений в узле родителе
                                   min_samples_split=10,
                                   # число узлов в дереве (в RandomForest иначе !!!!!!!!!!!!!)
                                   max_depth=5,
                                   # как выдавать промежуточные результаты
                                   verbose=0
                                   # Есть и другие параметры, уменьшающие размер дерева,
                                   # такие же как у DecisionTreeClassifier
                                   )


In [34]:
# Активируем поиск на решетке GridSearchCV
from sklearn.model_selection import GridSearchCV

param_grid = {  
      'n_estimators': [500, 650, 800],
      'max_depth': [4, 5, 6],
      'min_samples_leaf': [3, 5, 7],
      'min_samples_split': [5, 10, 15]
  }

In [35]:
model2 = GradientBoostingClassifier(random_state=42, subsample=0.66,min_impurity_decrease=0.001, criterion='friedman_mse',  learning_rate=0.01, loss='deviance',max_features='sqrt')
grid_search_res = GridSearchCV(model2, param_grid=param_grid, cv=2, error_score='raise', 
                               pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [36]:
grid_search_res.fit(X_train, y_train)

GridSearchCV(cv=2, error_score='raise',
             estimator=GradientBoostingClassifier(learning_rate=0.01,
                                                  max_features='sqrt',
                                                  min_impurity_decrease=0.001,
                                                  random_state=42,
                                                  subsample=0.66),
             param_grid={'max_depth': [4, 5, 6], 'min_samples_leaf': [3, 5, 7],
                         'min_samples_split': [5, 10, 15],
                         'n_estimators': [500, 650, 800]},
             scoring='accuracy')

In [46]:
grid_search_res.cv_results_

{'mean_fit_time': array([1.62408662, 2.02785683, 2.93787909, 1.9294498 , 2.4152447 ,
        2.99205756, 1.7583859 , 2.33324444, 2.73238742, 1.89426303,
        2.44054067, 3.09656405, 1.92485571, 2.76759005, 2.97482359,
        1.59935915, 1.81444681, 2.55577707, 1.71117425, 2.01954317,
        3.26435888, 1.99203873, 2.77955997, 2.58000612, 1.8163898 ,
        2.18785965, 2.32226241, 1.86171114, 2.39378834, 2.93855882,
        1.74161541, 2.24928796, 2.79971731, 1.88651288, 2.56972897,
        3.47532797, 2.16115046, 2.8057574 , 3.43598866, 2.09996891,
        2.82747281, 3.5182544 , 1.77187955, 2.35698831, 3.24894845,
        2.01232982, 2.61964309, 3.29954112, 2.19283116, 2.44071043,
        2.79093707, 1.84119022, 2.45271862, 3.1718272 , 2.77620625,
        3.63560665, 3.63098311, 2.46829343, 3.24773598, 3.99040091,
        2.07303405, 2.77382779, 2.96207511, 2.29166114, 2.63283312,
        3.84893322, 2.73288596, 3.35614324, 3.95870864, 2.4223609 ,
        3.19161892, 3.60807192,

In [38]:
# Строим предсказание модели
y_pred_train = grid_search_res.predict(X_train)
y_pred_test = grid_search_res.predict(X_test)

In [39]:
from sklearn import metrics
conf_mat = metrics.confusion_matrix(y_test, y_pred_test)
conf_mat = pd.DataFrame(conf_mat, index=grid_search_res.classes_, columns=grid_search_res.classes_)
conf_mat

,5,6,7
5,125,22,1
6,36,79,16
7,1,18,22


In [40]:
conf_mat = metrics.confusion_matrix(y_train, y_pred_train)
conf_mat = pd.DataFrame(conf_mat, index=grid_search_res.classes_, columns=grid_search_res.classes_)
conf_mat

,5,6,7
5,536,57,3
6,59,441,7
7,4,24,148


In [42]:
#Строим предсказание модели
y_pred_train2 = grid_search_res.predict_proba(X_train)
y_pred_test2 = grid_search_res.predict_proba(X_test)
part = sum(np.sum(y_pred_test2>0.8, axis =1))/np.shape(y_pred_test2)[0]*100
print('Доля наблюдений в тестовой выборке, для которых есть класс, вероятность принадлежать которому больше 0.8 равна',part,'%')

Доля наблюдений в тестовой выборке, для которых есть класс, вероятность принадлежать которому больше 0.8 равна 20.0 %


In [33]:
print("Accuracy on train set is ", metrics.accuracy_score(y_train, y_pred_train)*100)
print("Accuracy on test set is ", metrics.accuracy_score(y_test, y_pred_test)*100)

Accuracy on train set is  94.91790461297889
Accuracy on test set is  72.8125


In [43]:
y_pred_train2 = grid_search_res.predict(X_train)
y_pred_test2 = grid_search_res.predict(X_test)
print(metrics.classification_report(y_pred_test2, y_test))

              precision    recall  f1-score   support

           5       0.84      0.77      0.81       162
           6       0.60      0.66      0.63       119
           7       0.54      0.56      0.55        39

    accuracy                           0.71       320
   macro avg       0.66      0.67      0.66       320
weighted avg       0.72      0.71      0.71       320



In [44]:
best_CV_score = grid_search_res.best_estimator_
test_acc = best_CV_score.score(X_test,y_test)
# начение критерия качества для лучших значений гиперпараметров 
print('Лучшее accuracy на тестовом множестве', test_acc)

Лучшее accuracy на тестовом множестве 0.70625


## Калибровка

In [49]:
from sklearn.calibration import CalibratedClassifierCV

In [50]:
model_sigmoid = CalibratedClassifierCV(grid_search_res, cv=2, method='sigmoid')

In [51]:
model_sigmoid.fit(X_train, y_train)

CalibratedClassifierCV(base_estimator=GridSearchCV(cv=2, error_score='raise',
                                                   estimator=GradientBoostingClassifier(learning_rate=0.01,
                                                                                        max_features='sqrt',
                                                                                        min_impurity_decrease=0.001,
                                                                                        random_state=42,
                                                                                        subsample=0.66),
                                                   param_grid={'max_depth': [4,
                                                                             5,
                                                                             6],
                                                               'min_samples_leaf': [3,
                                                  

In [85]:
y_pred_train3 = model_sigmoid.predict(X_train)
y_pred_test3 = model_sigmoid.predict(X_test)
print(metrics.classification_report(y_pred_test3, y_test))

              precision    recall  f1-score   support

           5       0.87      0.75      0.81       171
           6       0.57      0.68      0.62       110
           7       0.51      0.54      0.53        39

    accuracy                           0.70       320
   macro avg       0.65      0.66      0.65       320
weighted avg       0.73      0.70      0.71       320



In [95]:
y_test_4 = model_sigmoid.predict(X_test)

In [96]:
part = sum(np.sum(y_train_4>0.8, axis =1))/np.shape(y_test_4)[0]*100
print('Доля наблюдений в тестовой выборке после калибровки, для которых есть класс, вероятность принадлежать которому больше 0.8 равна',part,'%')

Доля наблюдений в тестовой выборке после калибровки, для которых есть класс, вероятность принадлежать которому больше 0.8 равна 34.6875 %
